# Генерация заголовков новостей с использованием Char-RNN
Идея Char-RNN описана в [статье Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

Первоначальная версия кода взята из [примера реализации Char-RNN на Keras](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py). 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras import backend as K
import numpy as np
import random
import time

Здесь задаем, на каких данных будем обучать сеть.

In [ ]:
# Все статьи
# mode = "full"       

# Выборка (~800 статей)
mode = "sample"  

Откроем файл с предварительно обработанным текстом (см. ноутбук [preprocessing](/preprocessing.ipynb)) и оценим количество слов:

In [ ]:
f = open('./data/headers_' + mode + '.txt', 'r', encoding='utf-8')
text = f.read().lower()
print('Размер корпуса:', len(text))

Получаем набор всех символов в тексте и составляем 2 словаря:
* `char_indices` содержит символы и соотвествующий им индекс
* `indices_char` позволит по индексу получить символ

In [ ]:
chars = sorted(list(set(text)))
print('Всего символов:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Вспомогательная функция, выдающая случайный символ из наиболее вероятных. `temperature` отвечает за вариативность выдаваемых индексов: более низкое значение будет выдавать более вероятный символ с меньшей вариативностью, и наоборот.

На вход функции подаются вероятности, предсказанные сетью, на выходе выдается индекс символа.

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Разрезаем текст на куски по `maxlen` символов с шагом `step`. Чем больше `maxlen`, тем более "глубокими" становятся зависимости последующих символов от предыдущих. Также, чем меньше `step`, тем больше различных комбинаций символов попадет в обучающие данные. 

При `maxlen` = 5 и `step` = 2 фраза "Ну что, как дела?" будет разрезана на:
* "Ну чт"
* " что "
* "то, ка"
* ", как "
и т.д.

In [ ]:
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Последовательностей:', len(sentences))

Преобразуем полученные последовательности символов в последовательности векторов, в которых каждому символу будет соотвествовать вектор [x1, x2, ..., xK, ... xN], где xK = 1, если K равно индексу данного символа, N = количество в нашем наборе `chars`, а все остальные значения равны нулю. 

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Задаем гиперпараметры:
* `epochs`: количество эпох (итераций) обучения;
* `batch_size`: размер мини-батча, чем он меньше, тем менее усредненной будет ошибка, соответственно, будет выше точность и меньше вероятность скатиться в локальный минимум;
* `options`: массив опций для обучения; используется для того, чтобы определить наиболее удачную конфигурацию нейронной сети; для обучения только по одному набору параметров, можно закомментировать остальные наборы;
* `rnn_size`: количество LSTM-ячеек;
* `rnn_layers`: количество LSTM-слоев;
* `gen_length`: длина генерируемого текста в символах. 

In [ ]:
epochs = 50
batch_size = 256
options = [
#     {
#         'rnn_size': 32,
#         'rnn_layers': 1
#     },
#     {
#         'rnn_size': 64,
#         'rnn_layers': 1
#     },
#     {
#         'rnn_size': 32,
#         'rnn_layers': 2
#     },
    {
        'rnn_size': 64,
        'rnn_layers': 2
    }
]
gen_length = 150

## Обучение
Для каждого набора параметров строим модель, обучаем ее за `epochs` итераций, после каждой итерации выводим сгенерированный текст, а также сохраняем его в файл. Кроме того, на каждой итерации сохраняем модель.

In [ ]:
for option in options:
    
    # Обнуляем сессию
    K.clear_session()
    print('Тренируем модель Char-RNN с параметрами (слоев: {}, размер: {})'.format(option['rnn_layers'], option['rnn_size']))
    
    # Сохраняем время начала обучения для этого набора параметров
    option['start'] = time.time()
    
    # Строим модель
    model = Sequential()
    if option['rnn_layers'] == 1:
        model.add(LSTM(option['rnn_layers'], input_shape=(maxlen, len(chars))))
    else:
        model.add(LSTM(option['rnn_layers'], input_shape=(maxlen, len(chars)), return_sequences=True))
        model.add(LSTM(option['rnn_layers']))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01, decay=1e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    # Результаты будут сохраняться в этот файл
    with open('results_char_rnn_{}_{}.txt'.format(option['rnn_layers'], option['rnn_size']), 'a') as file:
        
        # Цикл с эпохами
        for iteration in range(1, epochs):
            print()
            print('-' * 50)
            print('Эпоха ', iteration)

            # Здесь ставим 1 эпоху, т.к. за итерации отвечает верхний цикл
            # Это сделано для того, чтобы после каждой итерации можно было провести дополнительные операции,
            # а возиться с callback'ами не очень хотелось...
            h = model.fit(x, y,
                      batch_size=batch_size,
                      epochs=1,
                      verbose=2)

            # Случайно выбираем начало входной последовательности
            start_index = random.randint(0, len(text) - maxlen - 1)
            
            file.write('Эпоха ' + str(iteration) + '\n')
            file.write(str(h.history) + '\n')
            
            # Выводим результаты генерации с разными температурами (вариативностью)
            for diversity in [0.5, 1.0]:
                print()
                print('----- diversity:', diversity)
                file.write('----- diversity: ' +str(diversity) + '\n')
                generated = ''
                sentence = text[start_index: start_index + maxlen]
                generated += sentence
                print('----- Начальная последовательность: "' + sentence + '"')
                file.write('----- Начальная последовательность: "' + sentence + '"\n')
                file.write(generated)
                
                # Запускаем посимвольную генерацию
                for i in range(gen_length):
                    # Создаем массив нулей, в который запишем векторы символов входной строки
                    x_pred = np.zeros((1, maxlen, len(chars)))
                    for t, char in enumerate(sentence):
                        x_pred[0, t, char_indices[char]] = 1.
                    next_char = ""
                    
                    # На всякий случай делаем это в цикле, т.к. бывали случаи, когда генерируемый символ почему-то
                    # оказывался не из нашего набора (какой-то баг), в таком случае генерируем другой символ
                    while next_char not in chars:
                        preds = model.predict(x_pred, verbose=0)[0]
                        next_index = sample(preds, diversity)
                        next_char = indices_char[next_index]

                    # Добавляем новый символ к строке результата
                    generated += next_char
                    sentence = sentence[1:] + next_char
                
                print(generated + '\n')
                file.write(next_char)
                file.write('\n')
            
            file.write('------------------------------------------\n')

            # Тут считаем время, оставшееся до завершения обучения по текущему набору параметров
            t = time.time()
            time_diff = t - option['start']
            total_time = round((epochs / (iteration + 1)) * time_diff)
            rem_time = round(total_time - time_diff)
            m, s = divmod(rem_time, 60)
            h, m = divmod(m, 60)
            print("Оценка оставшегося времени на обучение по текущему набору параметров: %d:%02d:%02d " % (h, m, s), end="\r")

            # И сохраняем модель
            model.save("./models/char_rnn_{}_{}.h5".format(option['rnn_layers'], option['rnn_size']))

        # В конце считаем время, затраченное на обучение по текущему набору параметров
        time_diff = time.time() - option['start']
        m, s = divmod(time_diff, 60)
        h, m = divmod(m, 60)

        print("Затраченное время: %d:%02d:%02d " % (h, m, s))
        file.write("Затраченное время: %d:%02d:%02d " % (h, m, s))
        print('Обучение модели Char-RNN с параметрами {} {} завершено'.format(option['rnn_layers'], option['rnn_size']))